### Idea Working Towards:
- Load well with LASIO
- Export LASIO style JSON
- CONVERT LASIO style JSON to WELLIO style JSON
- Visualize with wellioviz

In [24]:
import json

In [4]:
#!pip install matplotlib
##!pip install pixiedust_node

ABOUT PIXIEDUST NODE
WHICH LETS YOU RUN NODE.js CODE IN JUPYTER
https://github.com/pixiedust/pixiedust_node

In [6]:
import pixiedust_node

Pixiedust database opened successfully


In [13]:
%%node
//// THIS IS A JavaScript CELL
var str1 = "hello world";
var n1 = 4.1515;
var n2 = 42;
var tf = true;
var obj = { name:"Frank", age: 42 };
var array_of_strings = ["hello", "world"];
var array_of_objects = [{a:1,b:2}, {a:3, b:4}];

In [14]:
#### THIS IS A PYTHON CELL
print(str1, n1, n2, tf)
print(obj)
print(array_of_strings)
print(array_of_objects)

hello world 4.1515 42 True
{'name': 'Frank', 'age': 42}
['hello', 'world']
[{'a': 1, 'b': 2}, {'a': 3, 'b': 4}]


Geting wellio.js

In [7]:
npm.install('wellio')

/usr/bin/npm install -s wellio
pixiedust_node 0.2.5 started. Cells starting '%%node' may contain Node.js code.
+ wellio@0.1.6
updated 1 package and audited 209 packages in 1.29s
found 0 vulnerabilities


In [8]:
%%node
var wellio = require('wellio');

test of wellio

In [9]:
%%node
console.log(wellio.returnThing("asdfa"))

asdfa


Now lets get lasio

In [15]:
#!pip install lasio

In [16]:
import lasio

In [17]:
import numpy as np
from datetime import datetime

In [18]:
datetime.today().strftime('%Y-%m-%d %H:%M:%S')

'2020-02-15 22:40:00'

In [19]:
las = lasio.LASFile()

las.well.DATE = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
las.params['ENG'] = lasio.HeaderItem('ENG', value='Kent Inverarity')
las.params['LMF'] = lasio.HeaderItem('LMF', value='GL')
las.other = 'Example of how to create a LAS file from scratch using lasio'
depths = np.arange(10, 50, 0.5)
synth = np.log10(depths)*5+np.random.random(len(depths))
synth[:8] = np.nan
las.add_curve('DEPT', depths, unit='m')
las.add_curve('SYNTH', synth, descr='fake data')
las.write('scratch_v1.2.las', version=1.2)
las.write('scratch_v2.las', version=2)

In [20]:
las

In [21]:
las.curves

[CurveItem(mnemonic=DEPT, unit=m, value=, descr=, original_mnemonic=DEPT, data.shape=(80,)),
 CurveItem(mnemonic=SYNTH, unit=, value=, descr=fake data, original_mnemonic=SYNTH, data.shape=(80,))]

In [25]:
json_images = json.dumps(las, cls=lasio.JSONEncoder)

In [26]:
json_images

'{"metadata": {"Version": [{}, {}, {}], "Well": [{}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}], "Curves": [{}, {}], "Parameter": [{}, {}], "Other": "Example of how to create a LAS file from scratch using lasio"}, "data": {"DEPT": [10.0, 10.5, 11.0, 11.5, 12.0, 12.5, 13.0, 13.5, 14.0, 14.5, 15.0, 15.5, 16.0, 16.5, 17.0, 17.5, 18.0, 18.5, 19.0, 19.5, 20.0, 20.5, 21.0, 21.5, 22.0, 22.5, 23.0, 23.5, 24.0, 24.5, 25.0, 25.5, 26.0, 26.5, 27.0, 27.5, 28.0, 28.5, 29.0, 29.5, 30.0, 30.5, 31.0, 31.5, 32.0, 32.5, 33.0, 33.5, 34.0, 34.5, 35.0, 35.5, 36.0, 36.5, 37.0, 37.5, 38.0, 38.5, 39.0, 39.5, 40.0, 40.5, 41.0, 41.5, 42.0, 42.5, 43.0, 43.5, 44.0, 44.5, 45.0, 45.5, 46.0, 46.5, 47.0, 47.5, 48.0, 48.5, 49.0, 49.5], "SYNTH": [NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, 6.341544096786395, 6.567376614285353, 6.245335149382308, 6.448556279813931, 6.671890205581146, 6.241101400309921, 6.981955840011141, 6.646269547490197, 6.790654415948515, 6.6091616918326475, 7.173010371410631, 6.81162221

In [27]:
las_json_dict =json.loads(json_images)

In [28]:
with open('data.json', 'w') as outfile:
    json.dump(las_json_dict, outfile)

In [29]:
!ls

data.json  scratch_v1.2.las  Untitled1.ipynb  wellio.js and lasio test.ipynb
LICENSE    scratch_v2.las    Untitled.ipynb


In [30]:
%%node
wellio.read_json('data.json')

Thrown:
{ Error: ENOENT: no such file or directory, open 'data.json'
at Object.openSync (fs.js:443:3)
at Object.readFileSync (fs.js:343:35)
at Object.read_json (/home/jgosses82/node/node_modules/wellio/dist/index.js:17:28) errno: -2, syscall: 'open', code: 'ENOENT', path: 'data.json' }


In [35]:
%%node
var fs = require('fs');

In [48]:
%%node
var lj2 = las_json_dict

Apparently the node magic does some weird thing that makes it cumbersome to overwrite variable names? or something? so for testing purposes I had to re-write the lj variable into something different that it hadn't seen before to get the function below to work, hence, lj3

In [63]:
%%node
delete lj

#### THE fs to upload the json file doesn't work well for some reason in this setting, 
#### so switched to just grabbing the a json dict I made from the lassio json in string form
#### I still used "most" of the read_json function

In [64]:
%%node


    var lj3 = las_json_dict
    std_headers = {
      'Version': 'VERSION INFORMATION',
      'Well': 'WELL INFORMATION BLOCK',
      'Curves': 'CURVE INFORMATION BLOCK', 
      'Parameter': 'PARAMETER INFORMATION'
    };

    lasjson = {};
    lasjson["VERSION INFORMATION"] = {};
    lasjson["WELL INFORMATION BLOCK"] = {};
    lasjson["CURVE INFORMATION BLOCK"] = {};
    lasjson["PARAMETER INFORMATION"] = {};
    lasjson["CURVES"] = lj3.data;

    for (item in lj3.metadata) {
      if ( !(item in std_headers) ) {
        lasjson[item.toUpperCase()] = lj3.metadata[item];
      }
      else {
        for (var mnemonic in lj3.metadata[item]) {
          section = std_headers[item];      
          lasjson[section][mnemonic] = {
            MNEM: mnemonic,
            UNIT: '',
            DATA: lj3.metadata[item][mnemonic],
            'DESCRIPTION OF MNEMONIC 1': '',
            'DESCRIPTION OF MNEMONIC 2': ''
          };
        }
      }
    }

... ... ... ... ...
... ..... ..... ... ..... ....... ....... ......... ......... ......... ......... ......... ......... ....... ..... ...


In [65]:
%%node
console.log(lasjson)

{ 'VERSION INFORMATION':
{ '0':
{ MNEM: '0',
UNIT: '',
DATA: {},
'DESCRIPTION OF MNEMONIC 1': '',
'DESCRIPTION OF MNEMONIC 2': '' },
'1':
{ MNEM: '1',
UNIT: '',
DATA: {},
'DESCRIPTION OF MNEMONIC 1': '',
'DESCRIPTION OF MNEMONIC 2': '' },
'2':
{ MNEM: '2',
UNIT: '',
DATA: {},
'DESCRIPTION OF MNEMONIC 1': '',
'DESCRIPTION OF MNEMONIC 2': '' } },
'WELL INFORMATION BLOCK':
{ '0':
{ MNEM: '0',
UNIT: '',
DATA: {},
'DESCRIPTION OF MNEMONIC 1': '',
'DESCRIPTION OF MNEMONIC 2': '' },
'1':
{ MNEM: '1',
UNIT: '',
DATA: {},
'DESCRIPTION OF MNEMONIC 1': '',
'DESCRIPTION OF MNEMONIC 2': '' },
'2':
{ MNEM: '2',
UNIT: '',
DATA: {},
'DESCRIPTION OF MNEMONIC 1': '',
'DESCRIPTION OF MNEMONIC 2': '' },
'3':
{ MNEM: '3',
UNIT: '',
DATA: {},
'DESCRIPTION OF MNEMONIC 1': '',
'DESCRIPTION OF MNEMONIC 2': '' },
'4':
{ MNEM: '4',
UNIT: '',
DATA: {},
'DESCRIPTION OF MNEMONIC 1': '',
'DESCRIPTION OF MNEMONIC 2': '' },
'5':
{ MNEM: '5',
UNIT: '',
DATA: {},
'DESCRIPTION OF MNEMONIC 1': '',
'DESCRIPTION OF MNEMONIC

### and it works!

## We should probably write two read LASIO json functions to separate the file system upload from the one that parses the dict... That way people can either (1) upload from JSON file in lasio style of JSON (2) start from lasio style json string (3) start from lasio style dict